In [ ]:
import pandas as pd
from difflib import get_close_matches as gcm

all_skills = 'skills/Skills.xlsx'
digital_skills = 'skills/Digital Skills.xlsx'
coursera_skills = 'coursera_skills/Coursera Skills To DB.xlsx'

In [2]:
def extract_skills(file_name):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    all_skills = []
    for key, df in dfs.items():
        all_skills.extend(df['Skill'].unique().tolist())
    df_skills = pd.DataFrame({'skill': all_skills})
    return df_skills.drop_duplicates()

def parent_check(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    for key, df in dfs.items():
        df_error = df.loc[~df['Parent'].isin(df_skills['skill'])].dropna()
        print(df_error['Parent'].unique().tolist())

def extract_relation(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    df_rel = pd.DataFrame(columns=['Skill', 'Parent'])
    skill_to_id_dict = df_skills.set_index('skill').to_dict()['skill_id']
    for key, df in dfs.items():
        df_rel = df_rel.append(df.replace(skill_to_id_dict))
    df_rel = df_rel.reset_index(drop=True)
    df_rel = df_rel.dropna()
    df_rel = df_rel.drop_duplicates()
    df_rel['Parent'] = df_rel['Parent'].astype(int)
    return df_rel

### Extract Skills

In [3]:
# All skills
df_s1 = extract_skills(all_skills)
df_s1.head()

,skill
0,Applied Science
1,Business
2,Computer Science
3,Data Science
4,Humanities


In [4]:
df_s1.describe(include='object')

,skill
count,2261
unique,2261
top,Mercurial
freq,1


In [5]:
# Digital skills
df_s2 = extract_skills(digital_skills)
df_s2.head()

,skill
0,Algebra
1,Algorithm
2,Analytics
3,Artificial Intelligence (AI)
4,Big Data


In [6]:
df_s2.describe(include='object')

,skill
count,1009
unique,1009
top,Keras
freq,1


In [7]:
# Combine
df_skills = df_s1.append(df_s2)
df_skills = df_skills.drop_duplicates()
df_skills = df_skills.dropna()
df_skills.describe()

,skill
count,3066
unique,3066
top,Mercurial
freq,1


In [8]:
df_skills = df_skills.reset_index(drop=True)
df_skills['skill_id'] = df_skills.index + 1
# df_skills['skill_id'] = df_skills.groupby('skill').grouper.group_info[0] + 1
# df_skills = df_skills.reset_index(drop=True)
df_skills = df_skills[['skill_id', 'skill']]
df_skills.head()

,skill_id,skill
0,1,Applied Science
1,2,Business
2,3,Computer Science
3,4,Data Science
4,5,Humanities


In [9]:
df_skills.describe(include='all')

,skill_id,skill
count,3066.000000,3066
unique,NaN,3066
top,NaN,Mercurial
freq,NaN,1
mean,1533.500000,NaN
std,885.222288,NaN
min,1.000000,NaN
25%,767.250000,NaN
50%,1533.500000,NaN
75%,2299.750000,NaN


### Coursera Skills

In [10]:
df_c = pd.read_excel(coursera_skills)
df_c.head()

,skill_id,c_skill,Skill,Parent
0,457,Low Poly,Low Poly,3D Computer Graphics
1,462,Polygon Mesh,Polygon Mesh,3D Computer Graphics
2,466,Uv Mapping,Uv Mapping,3D Computer Graphics
3,467,Framing,Framing,3D Computer Graphics
4,469,3D Graphics Software,3D Graphics Software,3D Computer Graphics


In [11]:
df_coursera = df_c[['skill_id', 'c_skill']]
df_coursera = df_coursera.rename(columns={'c_skill': 'skill'})
df_coursera.head()

,skill_id,skill
0,457,Low Poly
1,462,Polygon Mesh
2,466,Uv Mapping
3,467,Framing
4,469,3D Graphics Software


In [12]:
df_others = df_c[['Skill']].rename(columns={'Skill': 'skill'})
df_others = df_others.drop_duplicates()
df_others = df_others.dropna()
df_others['skill_id'] = df_others.index + df_skills['skill_id'].max()
df_others.head()

,skill,skill_id
0,Low Poly,3066
1,Polygon Mesh,3067
2,Uv Mapping,3068
3,Framing,3069
4,3D Graphics Software,3070


In [13]:
# Combine
df_all = df_skills.append(df_others)
df_all = df_all.drop_duplicates(subset=['skill'])
df_all = df_all.reset_index(drop=True)
df_all = df_all[['skill_id', 'skill']]
df_all.head()

,skill_id,skill
0,1,Applied Science
1,2,Business
2,3,Computer Science
3,4,Data Science
4,5,Humanities


In [14]:
df_all.describe(include='all')

,skill_id,skill
count,9471.000000,9471
unique,NaN,9471
top,NaN,GreenQloud
freq,NaN,1
mean,5156.899060,NaN
std,3076.440191,NaN
min,1.000000,NaN
25%,2368.500000,NaN
50%,5260.000000,NaN
75%,7882.000000,NaN


### Data (Parent) Checking/Cleaning

In [15]:
parent_check(all_skills, df_skills)

[]
[]
[]
[]
[]
[]
[]
[]
[]


In [16]:
parent_check(digital_skills, df_skills)

[]
[]
[]
[]
[]
[]


In [17]:
parent_check(coursera_skills, df_all)

[]


### Data (Duplicate) Checking/Cleaning

In [ ]:
def skill_dup_check(reverse=False):
    skill_list = df_skills['skill'].tolist()
    pos_dups = {}
    for i in range(len(skill_list)):
        if reverse:
            skill = skill_list[-(i+1)]
            remaining = skill_list[:-(i+1)]
        else:
            skill = skill_list[i]
            remaining = skill_list[i+1:]
        pos_dup = gcm(skill, remaining, cutoff=0.9)
        if len(pos_dup) > 0:
            pos_dups[skill] = pos_dup
    return pos_dups

In [ ]:
pos_dups = skill_dup_check()

for s in pos_dups:
    print('{}: {}'.format(s, pos_dups[s]))

In [ ]:
pos_dups = skill_dup_check(reverse=True)

for s in pos_dups:
    print('{}: {}'.format(s, pos_dups[s]))

### Extract Skill Relation

In [18]:
# Skills
df_r1 = extract_relation(all_skills, df_skills)
df_r2 = extract_relation(digital_skills, df_skills)
df_rel = df_r1.append(df_r2)
df_rel = df_rel.drop_duplicates()
df_rel = df_rel.reset_index(drop=True)
df_rel = df_rel.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel.head()

,skill_1_id,skill_2_id
0,11,2
1,12,2
2,13,74
3,14,74
4,15,1


In [19]:
df_rel.describe(include='all')

,skill_1_id,skill_2_id
count,3770.0,3770.000000
unique,3056.0,NaN
top,121.0,NaN
freq,7.0,NaN
mean,NaN,846.392573
std,NaN,932.304624
min,NaN,1.000000
25%,NaN,105.000000
50%,NaN,439.000000
75%,NaN,1481.000000


In [20]:
# Include coursera skills
df_r3 = extract_relation(coursera_skills, df_all)
# Combine
df_rel_all = df_rel.append(df_r3[['Skill', 'Parent']])
df_rel_all = df_rel_all.drop_duplicates()
df_rel_all = df_rel_all.reset_index(drop=True)
df_rel_all = df_rel_all.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel_all.head()

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
0,11,2.0,NaN,NaN
1,12,2.0,NaN,NaN
2,13,74.0,NaN,NaN
3,14,74.0,NaN,NaN
4,15,1.0,NaN,NaN


In [21]:
df_rel_all.describe(include='all')

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
count,3770.0,3770.000000,2349.0,2349.000000
unique,3056.0,NaN,2343.0,NaN
top,121.0,NaN,1130.0,NaN
freq,7.0,NaN,2.0,NaN
mean,NaN,846.392573,NaN,907.340145
std,NaN,932.304624,NaN,1429.162846
min,NaN,1.000000,NaN,1.000000
25%,NaN,105.000000,NaN,80.000000
50%,NaN,439.000000,NaN,158.000000
75%,NaN,1481.000000,NaN,1192.000000


### Extract Coursera Relation

In [22]:
df_crel = df_c[['skill_id', 'Skill']].rename(columns={'skill_id': 'coursera_skill_id', 'Skill': 'skill'})\
    .merge(df_all, how='left').drop(columns=['skill'])
df_crel.head()

,coursera_skill_id,skill_id
0,457,3066
1,462,3067
2,466,3068
3,467,3069
4,469,3070


In [23]:
df_crel.describe(include='all')

,coursera_skill_id,skill_id
count,7301.000000,7301.000000
mean,3650.941652,6218.926174
std,2107.856673,2672.374568
min,0.000000,2.000000
25%,1826.000000,4267.000000
50%,3651.000000,6475.000000
75%,5476.000000,8465.000000
max,7301.000000,10366.000000


### No Parent Check

In [24]:
df_st = df_rel.replace(df_skills.set_index('skill_id').to_dict()['skill'])\
    .rename(columns={'skill_1_id': 'skill', 'skill_2_id': 'parent'})
no_parents = []

for skill in df_skills['skill'].tolist():
    if skill not in df_st['skill'].tolist():
        no_parents.append(skill)

no_parents

['Applied Science',
 'Business',
 'Computer Science',
 'Data Science',
 'Humanities',
 'Information Technology',
 'Language Learning',
 'Mathematics',
 'Natural Science',
 'Social Science']

In [28]:
print(no_parents)

['Applied Science', 'Business', 'Computer Science', 'Data Science', 'Humanities', 'Information Technology', 'Language Learning', 'Mathematics', 'Natural Science', 'Social Science']


In [30]:
def check_sheet(file_name, to_check):
    xl_file = pd.ExcelFile(file_name)
    for sheet_name in xl_file.sheet_names:
        df = xl_file.parse(sheet_name)
        for s in to_check:
            if s in df['Skill'].tolist():
                print(s, ':', sheet_name)

In [31]:
to_check = ['Rstudio']

check_sheet(all_skills, to_check)
print()
check_sheet(digital_skills, to_check)

Rstudio : Courses

Rstudio : CS


### Export

In [29]:
df_skills.to_csv('skills_db/skill.csv', index=False)
df_rel.to_csv('skills_db/skill_tree.csv', index=False)
df_all.to_csv('skills_db/all_skill.csv', index=False)
df_rel_all.to_csv('skills_db/all_skill_tree.csv', index=False)
df_coursera.to_csv('skills_db/coursera_skill.csv', index=False)
df_crel.to_csv('skills_db/coursera_map.csv', index=False)